# Data Analysis Quickstart


## Introduction

Here we'll do a whistlestop tour of data analysis on a *single* dataframe using the Star Wars' characters dataset as an example. For a more thorough grounding in using data, see the next chapter.

This chapter uses the **pandas**, **pyarrow**, and **numpy** packages. If you're running this code, you may need to install these packages, which you can do using either `conda install packagename` or `pip install packagename` on your computer's command line. (If you're not sure what a command line or terminal is, take a quick look at the basics-of-coding chapter.)

This chapter is hugely indebted to the fantastic [Python Data Science Handbook](https://jakevdp.github.io/PythonDataScienceHandbook/), and both the **pandas** [documentation](https://pandas.pydata.org/docs/user_guide/index.html) and amazing [introductory tutorials](https://pandas.pydata.org/pandas-docs/stable/getting_started/intro_tutorials/index.html).

## Loading data and checking datatypes

Loading data into a dataframe is achieved with commands like `df = pd.read_csv(...)` or `df = pd.read_stata(...)`. Let's load the Star Wars data:

In [ ]:
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

# Set seed for reproducibility
np.random.seed(10)
# Set max rows displayed for readability
pd.set_option('display.max_rows', 6)
# Plot settings
plt.style.use("https://github.com/aeturrell/coding-for-economists/raw/main/plot_style.txt")

df = pd.read_csv("https://github.com/aeturrell/coding-for-economists/raw/main/data/starwars.csv", index_col=0)
# Check info about dataframe
df.info()

### Look at the first few rows with `head()`

In [ ]:
df.head()

## Filter rows and columns with conditions using `df.loc[condition(s) or row(s), column(s)]`

`.loc` stands for location and allows you to filter (aka subset) a dataframe. `.loc` works like an index, so it always comes with square brackets, eg `df.loc[...]`.

`loc` takes two arguments. The first is a list of the names of the rows that you'd like to select *or* a condition (ie a list of booleans with the same length as the dataframe) that selects certain rows. Remember, you can easily create a series of booleans by checking a column against a condition, for example `df['column1'] == 'black'`.

The second argument consists of a list of column names you'd like to select. In both cases, `:` is shorthand for 'use all rows' or 'use all columns'. If you have *either* condition(s) *or* column(s) (but not both), you can simply write `df[condition(s)]` or `df[column(s)]`.

Here's an example with a condition built up out of two parts and a list of columns:

In [ ]:
df.loc[(df['hair_color'] == 'brown') & (df['eye_color'] == 'blue'), ['name', 'species']]

## Sort rows or columns with `.sort_values()`

Use `sort_values(columns, ascending=False)` for descending order.

In [ ]:
df.sort_values(['height', 'mass'])

## Choose multiple rows or columns using slices

Slices can be passed by name using `.loc[startrow:stoprow:step, startcolumn:stopcolumn:step]` or by position using `.iloc[start:stop:step, start:stop:step]`.

Choosing every 10th row from the second, and the columns between 'name' and 'gender':

In [ ]:
df.loc[2::10, 'name':'gender']

Note that `loc` only works here with numbers for rows because it just so happens that the names of the rows *are* numbers. If the rows had names that were strings, and we wanted to subset rows by their index position, we would have to use `iloc` instead.

Choosing the first 5 rows and the last 2 columns by index position:

In [ ]:
df.iloc[:5, -2:]

## Randomly selecting a sample using `.sample`

`.sample(n)` randomly selects `n` rows, `.sample(frac=0.4)` selects 40% of the data, `replace=True` samples with replacement, and passing `weights=` selects a number or fraction with the probabilities given by the passed weights. (Note that weights passed should have the same length as the dataframe.)

Taking a sample of 5 rows:

In [ ]:
df.sample(5)

## Rename with `.rename`

You can rename all columns by passing a function, for instance `df.rename(columns=str.lower)` to put all columns in lower case. Alternatively, use a dictionary to say which columns should be mapped to what:

In [ ]:
df.rename(columns={'homeworld': 'home_world'})

## Add new columns with `.assign` or assignment

Very often you will want to create new columns based on existing columns.

![](https://pandas.pydata.org/docs/_images/05_newcolumn_1.svg)

`.assign` is a function that applies to a dataframe and returns a copy of that dataframe:

In [ ]:
df.assign(height_m = df['height']/100)

This was added to the end; ideally, we'd like it next to the height column:

In [ ]:
(df.assign(height_m = df['height']/100)
   .sort_index(axis=1))

To overwrite existing columns just use `height = df['height']/100` in the assign function. We can also assign using a series of statements instead of chaining them together using

```python
df['height_m'] = df['height']/100
df = df.sort_index(axis=1)
```

## Summarise numerical values with `.describe()`



In [ ]:
df.describe()

## Group variables values with `.groupby()`



In [ ]:
df.groupby('species')[['height', 'mass']].mean()

## Add transformed columns using `.transform()`

Quite often, it's useful to put a column into a dataframe that is the result of an intermediate groupby and aggregation. For example, subtracting the group mean or normalisation. Transform does this and returns the column with the same shape as the original dataframe.

Below is an example of transform being used to demean a variable according to the mean by species. Note that we use `pd.notna` to filter out missing species values (otherwise this would result in an error, which is actually *helpful* behaviour) and we are using lambda functions. Lambda functions are a quick way of writing functions, e.g. `lambda x: x+1` defines a function that adds one to x.

In [ ]:
# First take a cut of the dataframe with no NA values in the species column:
df_cut = df[pd.notna(df['species'])]
# Now create a demeaned mass col
(df_cut
 .assign(mass_demean_species = df_cut
                               .groupby('species')['mass']
                               .transform(lambda x: x-x.mean()))
)

## Make quick charts with `.plot.*`

Including scatter, area, bar, box, density, hexbin, histogram, kde, and line.

In [ ]:
df.plot.scatter('mass', 'height', alpha=0.5);

In [ ]:
df.plot.box('species');

In [ ]:
df['height'].plot.kde(bw_method=0.3);

## Export results and descriptive statistics to latex with `to_latex`

You'll often want to export your results to a latex file for inclusion in a paper, presentation, or poster. Let's say we had some descriptive statistics on a dataframe:

In [ ]:
table = df[['mass', 'height']].agg([np.mean, np.std])
table

In [ ]:
print(table.to_latex(caption='A Table', label='tab:descriptive'))

Writing to the terminal isn't that useful for getting your paper done! To export to a file, use `table.to_latex('file.tex', ...)`.